### Choose the first entry of each patient. Extract body Pao2/fio2 ratio data into ***Pao2fio2***, create ***Pao2fio2_index*** for each patient. 
Contains patient body Pao2/fio2 ratio data from both vitalPeriodic.csv & nurseCharting.csv

**1. Extract sub-categories patient id from cardiovascular**

In [7]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# change to your folder path
os.chdir('C:/Users/xiao-zy19/Desktop/Johns Hopkins/Biomedical Data Design/EICU Database/eicu-collaborative-research-database-demo-2.0.1')
# os.chdir('/Users/xiao-zy19/Desktop/Johns Hopkins/Biomedical Data Design/DataExtract/data') 

# import diagnosis.csv
df_diagnosis = pd.read_csv('diagnosis.csv')
df_diagnosis.sort_values(by=['patientunitstayid', 'diagnosisoffset'], inplace=True)

# select cardiovascular patients
df_cardiovascular = df_diagnosis[df_diagnosis['diagnosisstring'].str.contains('cardiovascular')]

# get shock patient 
shock_patient = df_cardiovascular[df_cardiovascular['diagnosisstring'].str.contains('shock')]

# get ventricular patient 
ventricular_patient = df_cardiovascular[df_cardiovascular['diagnosisstring'].str.contains('ventricular')]

# get chest pain patient 
chest_pain_patient = df_cardiovascular[df_cardiovascular['diagnosisstring'].str.contains('chest pain')]

# get arrhythmias patient 
arrhythmias_patient = df_cardiovascular[df_cardiovascular['diagnosisstring'].str.contains('arrhythmias')]

# put id together
df_wanted = pd.concat([shock_patient, ventricular_patient, chest_pain_patient, arrhythmias_patient])

# Get the patient ids from df_wanted & sort the patient id
# patient_id_all multiple entry patient's stayid
patient_id_all = df_wanted['patientunitstayid'].unique()
patient_id_all.sort()
print(patient_id_all)

[ 143870  151179  151900 ... 3351297 3352230 3352231]


**2. Exclude patient whose unitvisitnumbe>1**

In [8]:
# import patient.csv
df_patient = pd.read_csv('patient.csv')
df_patient.sort_values(by=['patientunitstayid'], inplace=True)
df_patient_buf = df_patient[df_patient['patientunitstayid'].isin(patient_id_all)]
df_1time_patient = df_patient_buf[df_patient_buf['unitvisitnumber']==1]
# print(df_1time_patient)

# select the patient id from df_1time_patient
patient_id = df_1time_patient['patientunitstayid'].unique()
print(f'Total number of patients: {len(patient_id)}')

Total number of patients: 915


**3. Extract data of patients within the id list**

In [9]:
# import nurseCharting, lab, respiratoryCharting
df_nurseCharting = pd.read_csv('nurseCharting.csv')
df_nurseCharting.sort_values(by=['patientunitstayid', 'nursingchartoffset'], inplace=True)
df_lab = pd.read_csv('lab.csv')
df_lab.sort_values(by=['patientunitstayid', 'labresultoffset'], inplace=True)
df_respiratoryCharting = pd.read_csv('respiratoryCharting.csv')
df_respiratoryCharting.sort_values(by=['patientunitstayid', 'respchartoffset'], inplace=True)


# select the wanted patient
df_nurseCharting = df_nurseCharting[df_nurseCharting['patientunitstayid'].isin(patient_id)]
df_lab = df_lab[df_lab['patientunitstayid'].isin(patient_id)]
df_respiratoryCharting = df_respiratoryCharting[df_respiratoryCharting['patientunitstayid'].isin(patient_id)]


# print the shape of the wanted file
print(f'nurseCharting shape: {df_nurseCharting.shape}')
print(f'lab shape: {df_lab.shape}')
print(f'respiratoryCharting shape: {df_respiratoryCharting.shape}')

nurseCharting shape: (603027, 8)
lab shape: (182390, 10)
respiratoryCharting shape: (81833, 7)


**4. Extract Heart Rate data & create index**

In [13]:
# nursingchartcelltypevallabel: SVO2
df_nurseCharting_SVO = df_nurseCharting[df_nurseCharting['nursingchartcelltypevallabel']=='SVO2']
df_nurseCharting_SVO = df_nurseCharting_SVO.rename(columns={'nursingchartoffset': 'observationoffset', 'nursingchartvalue':'SVO2'})
# print(df_nurseCharting_SVO.head())

# nursingchartcelltypevallabel: O2 Saturation
df_nurseCharting_O2 = df_nurseCharting[df_nurseCharting['nursingchartcelltypevallabel']=='O2 Saturation']
df_nurseCharting_O2 = df_nurseCharting_O2.rename(columns={'nursingchartoffset': 'observationoffset', 'nursingchartvalue':'O2 Saturation'})
# print(df_nurseCharting_O2.head())

# labname: FiO2
df_lab_FiO2 = df_lab[df_lab['labname']=='FiO2']
df_lab_FiO2 = df_lab_FiO2.rename(columns={'labresultoffset': 'observationoffset', 'labresult':'FiO2'})
# print(df_lab_FiO2.head())

# labname: paO2
df_lab_paO2 = df_lab[df_lab['labname']=='paO2']
df_lab_paO2 = df_lab_paO2.rename(columns={'labresultoffset': 'observationoffset', 'labresult':'paO2'})
# print(df_lab_paO2.head())

In [11]:
# extract data
SVO2 = df_nurseCharting_SVO[['patientunitstayid', 'observationoffset', 'SVO2']]
O2 = df_nurseCharting_O2[['patientunitstayid', 'observationoffset', 'O2 Saturation']]

SVO2.sort_values(by=['patientunitstayid', 'observationoffset'], inplace=True)
O2.sort_values(by=['patientunitstayid', 'observationoffset'], inplace=True)



value_position_dict = {}
first_occurrences = []
for idx, value in enumerate(Temp['patientunitstayid']):
    # if the value is not in the dictionary, add it and create index
    if value not in value_position_dict:
        value_position_dict[value] = idx
        first_occurrences.append(idx)

first_occurrences.append(len(Temp))
# create first occurrence index for every patient
Temp_index = pd.Series(first_occurrences)
print(f'First 5 rows of Temp_index: \n{Temp_index.head()}')

NameError: name 'df_vitalPeriodic' is not defined

**Example: how to use Temp & Temp_index**

In [ ]:
# if we want the i th patient's data (i starts from 0)
# use Temp.iloc[Temp_index[i]:Temp_index[i+1]]
i = 0
print(f'HeartRate data for patient {i+1}: \n{Temp.iloc[Temp_index[i]:Temp_index[i+1]]}')